In [2]:
import numpy as np
import pandas as pd
import psycopg2
import os

In [3]:
cwd = os.getcwd()
path1 = os.path.join(cwd,r'data\08_12_chris_Balance_20190812T192339.csv')

#column_labels1 = ['X1','Y1','X2','Y2','X3','Y3','X4','Y4'] #determine column labels
experiment1 = pd.read_csv(path1,index_col = [0])
print(experiment1)

          AccelerometerX  AccelerometerY  AccelerometerZ  GyroscopeX  \
sequence                                                               
0                      2            -994              11          -8   
1                     36            -979              52          -9   
2                    -41            -986              32          -9   
3                      4            -972              29         -12   
4                     17            -987              17          -9   
...                  ...             ...             ...         ...   
9241                  -9            -983              39         -11   
9242                 -12            -979              16         -10   
9243                  -1            -975              33         -10   
9244                  -4           -1000              33         -10   
9245                 -35            -974              23          -9   

          GyroscopeY  GyroscopeZ  
sequence                    

In [4]:
    import psycopg2
    from io import StringIO

    # Connection parameters
    param_dic = {
        "host"      : "localhost",
        "database"  : "sensordata",
        "user"      : "postgres",
        "password"  : "JNt8hSW4l+MJ]p;.g[WG"
    }

    #Credit to Naysan Saran. https://naysan.ca/2020/06/21/pandas-to-postgresql-using-psycopg2-copy_from/

    def connect(params_dic):
        """ Connect to the PostgreSQL database server """
        conn = None
        try:
            # connect to the PostgreSQL server
            print('Connecting to the PostgreSQL database...')
            conn = psycopg2.connect(**params_dic)
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
            sys.exit(1) 
        print("Connection successful")
        return conn


    def copy_from_stringio(conn, df, table):
        """
        Here we are going save the dataframe in memory 
        and use copy_from() to copy it to the table
        """
        # save dataframe to an in memory buffer
        buffer = StringIO()
        df.to_csv(buffer, index_label='id', header=False)
        buffer.seek(0)

        cursor = conn.cursor()
        try:
            cursor.copy_from(buffer, table, sep=",")
            conn.commit()
        except (Exception, psycopg2.DatabaseError) as error:

            print("Error: %s" % error)
            conn.rollback()
            cursor.close()
            return 1
        print("copy_from_stringio() done")
        cursor.close()

In [5]:
conn = connect(param_dic) # connect to the database
cursor = conn.cursor() # set cursor

# CREATE TABLE
list1 = list(experiment1.keys()) #use column labels for SQL query
string1 = "CREATE TABLE SensiML (id int"
for label in list1:
    string1 += (", " + label + " decimal")
string1 = string1 + ");"
print(string1)
cursor.execute(string1) #execute SQL query

copy_from_stringio(conn, experiment1, "SensiML") # copy the dataframe to SQL
conn.close() # close the connection

Connecting to the PostgreSQL database...
Connection successful
CREATE TABLE SensiML (id int, AccelerometerX decimal, AccelerometerY decimal, AccelerometerZ decimal, GyroscopeX decimal, GyroscopeY decimal, GyroscopeZ decimal);
copy_from_stringio() done
